In [ ]:
#!/usr/bin/env python

from confluent_kafka import Consumer
from config import GetConsumerConfig
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroDeserializer
from config import GetProducerConfig

# if __name__ == '__main__':

config = GetConsumerConfig()



In [ ]:
class Message(object):

    def __init__(self, idade, name, team):
        self.name = name
        self.idade = idade
        self.team = team


In [ ]:
def convert_reverse(message,ctx):

    if message is None:
        return None

    return Message(idade=message['idade'],
                name=message['name'],
                team=message['team'])


schema_registry_conf = {'url': "https://@psrc-57o65q.us-west2.gcp.confluent.cloud",
                        'basic.auth.user.info': ":"
                        }
schema_registry_client = SchemaRegistryClient(schema_registry_conf)
schema = schema_registry_client.get_schema(100001)
avro_deserializer = AvroDeserializer(schema_registry_client,
                                     schema.schema_str,
                                     convert_reverse)


In [ ]:
# Create Producer instance
consumer = Consumer(config)


# Subscribe to topic

topic = "topic_0"
consumer.subscribe([topic])

# Poll for new messages from Kafka and print them.
try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:

            pass
        elif msg.error():
            print("ERROR: %s".format(msg.error()))
        else:
            message = avro_deserializer(msg.value(), SerializationContext(msg.topic(), MessageField.VALUE))
            if message is not None:
                print("User record {}: name: {}\n"
                      "\tidade: {}\n"
                      "\tmessage: {}\n"
                      .format(msg.key(), message.name,
                              message.idade,
                              message.team))
except KeyboardInterrupt:
    pass
finally:
    # Leave group and commit final offsets
    consumer.close()

In [ ]:

# Subscribe to topic
topic = "topic_0"
consumer.subscribe([topic])

# Poll for new messages from Kafka and print them.
try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            # Initial message consumption may take up to
            # `session.timeout.ms` for the consumer group to
            # rebalance and start consuming
            #print("Waiting...")
            pass
        elif msg.error():
            print("ERROR: %s".format(msg.error()))
        else:
            # Extract the (optional) key and value, and print.
            print("Consumed event from topic {topic}: key = {key:12} value = {value:12}".format(
                topic=msg.topic(), key=msg.key().decode('utf-8'), value=msg.value().decode('utf-8')))
except KeyboardInterrupt:
    pass
finally:
    # Leave group and commit final offsets
    consumer.close()